# **Mel Frequency Cepstral Coefficents (MFCCs)**
We follow along with this notebook: https://www.kaggle.com/code/ilyamich/mfcc-implementation-and-tutorial

Here we compute the MFCCs for a subset of the Xeno Canto bird songs dataset. This gives us the high-dimensional representation of the birds songs. From there, we compute the 3D UMAP embedding to create the 3D point cloud.

In [ ]:
!pip install pydub
!pip install psutil

In [ ]:
import os, psutil
import numpy as np
import pandas as pd
import scipy
from scipy.io import wavfile
import pydub
from pydub import AudioSegment
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import time

%matplotlib inline

In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/Dataset/Audio/XenoCanto/amazonas/birds/songs/'
pid = os.getpid()

In [ ]:
# do not run - just for backing up
# %cd ..
# %pwd

In [ ]:
# save all song names to output.txt
%ls /content/drive/MyDrive/Dataset/Audio/XenoCanto/amazonas/birds/songs | grep -vFx output.txt > output.txt

## Testing with One Song!

In [ ]:
# playing with just one song to get a sense of runtimes
sound = AudioSegment.from_mp3(TRAIN_PATH + "XC118422-ZIMGRA12.mp3")
sound.export("song_test.wav", format="wav")

<_io.BufferedRandom name='song_test.wav'>

In [ ]:
sample_rate, audio = wavfile.read("song_test.wav")
print("Sample rate: {0}Hz".format(sample_rate))
print("Audio duration: {0}s".format(len(audio) / sample_rate))

Sample rate: 44100Hz
Audio duration: 19.90530612244898s


In [ ]:
def normalize_audio(audio):
    audio = audio / np.max(np.abs(audio))
    return audio

In [ ]:
audio = normalize_audio(audio)
# plt.figure(figsize=(15,4))
# plt.plot(np.linspace(0, len(audio) / sample_rate, num=len(audio)), audio)
# plt.grid(True)

In [ ]:
def frame_audio(audio, FFT_size=2048, hop_size=10, sample_rate=44100):
    # hop_size in ms
    #print("inside frame audio!")

    #print("audio shape", audio.shape)
    #print("FFT_size / 2", int(FFT_size / 2))
    audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
    #print("1 passed audio")
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
    #print("2 passed frame_len")
    frame_num = int((len(audio) - FFT_size) / frame_len) + 1
    #print("3 passed frame_num")
    frames = np.zeros((frame_num,FFT_size))
    #print("4 passed frames")

    #print("frame_num", frame_num)
    for n in range(frame_num):
        frames[n] = audio[n*frame_len:n*frame_len+FFT_size]

    del frame_len
    del audio

    return frames

In [ ]:
hop_size = 15 #ms
FFT_size = 2048

audio_framed = frame_audio(audio, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate)
print("Framed audio shape: {0}".format(audio_framed.shape))

Framed audio shape: (1327, 2048)


In [ ]:
window = get_window("hann", FFT_size, fftbins=True)
# plt.figure(figsize=(15,4))
# plt.plot(window)
# plt.grid(True)

In [ ]:
audio_win = audio_framed * window

# ind = 69
# plt.figure(figsize=(15,6))
# plt.subplot(2, 1, 1)
# plt.plot(audio_framed[ind])
# plt.title('Original Frame')
# plt.grid(True)
# plt.subplot(2, 1, 2)
# plt.plot(audio_win[ind])
# plt.title('Frame After Windowing')
# plt.grid(True)

In [ ]:
audio_winT = np.transpose(audio_win)

audio_fft = np.empty((int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

for n in range(audio_fft.shape[1]):
    audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

audio_fft = np.transpose(audio_fft)

In [ ]:
audio_power = np.square(np.abs(audio_fft))
# print(audio_power.shape)

In [ ]:
freq_min = 0
freq_high = sample_rate / 2
mel_filter_num = 10

print("Minimum frequency: {0}".format(freq_min))
print("Maximum frequency: {0}".format(freq_high))

Minimum frequency: 0
Maximum frequency: 22050.0


In [ ]:
def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)

def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)

    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = met_to_freq(mels)

    return np.floor((FFT_size + 1) / sample_rate * freqs).astype(int), freqs

In [ ]:
filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=44100)
filter_points

array([   0,   12,   28,   51,   82,  125,  184,  265,  376,  528,  737,
       1024])

In [ ]:
def get_filters(filter_points, FFT_size):
    filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))

    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])

    return filters

In [ ]:
filters = get_filters(filter_points, FFT_size)

# plt.figure(figsize=(15,4))
# for n in range(filters.shape[0]):
#     plt.plot(filters[n])

In [ ]:
# taken from the librosa library
enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
filters *= enorm[:, np.newaxis]

In [ ]:
audio_filtered = np.dot(filters, np.transpose(audio_power))
audio_log = 10.0 * np.log10(audio_filtered)
audio_log.shape

(10, 1327)

In [ ]:
def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num,filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)

    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)

    return basis

In [ ]:
dct_filter_num = 40

dct_filters = dct(dct_filter_num, mel_filter_num)

cepstral_coefficents = np.dot(dct_filters, audio_log)
cepstral_coefficents.shape

(40, 1327)

In [ ]:
cepstral_coefficents[:, 0]

array([-2.72289196e+02, -4.63764728e+00, -4.52652370e+00, -3.60416039e+00,
       -8.04671288e-01, -4.37650817e+00, -8.47292766e-02,  9.48380847e-01,
        1.67173827e+00,  5.09526854e-01,  3.59427729e-13, -5.09526854e-01,
       -1.67173827e+00, -9.48380847e-01,  8.47292766e-02,  4.37650817e+00,
        8.04671288e-01,  3.60416039e+00,  4.52652370e+00,  4.63764728e+00,
        3.85075074e+02,  4.63764728e+00,  4.52652370e+00,  3.60416039e+00,
        8.04671288e-01,  4.37650817e+00,  8.47292766e-02, -9.48380847e-01,
       -1.67173827e+00, -5.09526854e-01, -5.70167271e-14,  5.09526854e-01,
        1.67173827e+00,  9.48380847e-01, -8.47292766e-02, -4.37650817e+00,
       -8.04671288e-01, -3.60416039e+00, -4.52652370e+00, -4.63764728e+00])

## Pipeline Module!
Here we generalize the above to run on a larger subset of the data.

In [ ]:
def computeMFCC(songName, filterNum = 40):
    start = time.time()
    # convert MP3 to WAV
    sound = AudioSegment.from_mp3(TRAIN_PATH + songName)
    wavFile = songName + ".wav"
    #print("wavfile shape", wavFile)
    #sound = sound.set_channels(1)
    sound.export(wavFile, format="wav")
    #print("WAV RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB


    # get the audio and sample rate
    sample_rate, audio = wavfile.read(wavFile)

    #print("Sample Rate: ", sample_rate)
    #print("Audio Length: ", len(audio) / sample_rate)
    #print("Audio Shape: ", audio.shape)
    if audio.ndim != 1:
        print("not mono", audio.ndim)
        audio = np.mean(audio, axis=1)
    # if len(audio) / sample_rate > 105:
    #     print("too long")
    #     return -1, -1
    del sound

    # normalize audio
    audio = normalize_audio(audio)
    #print("Audio Shape Normalized: ", audio.shape)
    #print("Normalize RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB

    # frame audio
    hop_size = 15 #ms
    FFT_size = 2048
    audio_framed = frame_audio(audio, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate)
    #print("Frame RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB

    window = get_window("hann", FFT_size, fftbins=True)
    audio_win = audio_framed * window
    del audio_framed
    del window
    audio_winT = np.transpose(audio_win)
    #print("Transpose RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB



    audio_fft = np.empty((int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

    for n in range(audio_fft.shape[1]):
        audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]
    del audio_winT

    audio_fft = np.transpose(audio_fft)
    audio_power = np.square(np.abs(audio_fft))
    del audio_fft
    #print("Power RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB


    freq_min = 0
    freq_high = sample_rate / 2
    mel_filter_num = 10
    filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=sample_rate)
    filters = get_filters(filter_points, FFT_size)
    #print("Filter RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB


    enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
    filters *= enorm[:, np.newaxis]
    del enorm

    audio_filtered = np.dot(filters, np.transpose(audio_power))
    del audio_power
    audio_log = 10.0 * np.log10(audio_filtered)
    del audio_filtered
    dct_filter_num = filterNum
    #print("DCT RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB


    dct_filters = dct(dct_filter_num, mel_filter_num)
    del dct_filter_num

    cepstral_coefficents = np.dot(dct_filters, audio_log)
    del dct_filters
    #print("DCT Filters RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB

    end = time.time()
    #print("Elapsed time: ", end - start)
    return cepstral_coefficents[:, 0], 1

In [ ]:
temp_dict = {}
coefficients = computeMFCC("XC118422-ZIMGRA12.mp3", filterNum = 512) #nice!
temp_dict["XC118422-ZIMGRA12.mp3"] = coefficients[0]

temp2 = pd.DataFrame(temp_dict).T
temp2

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
XC118422-ZIMGRA12.mp3,-272.289196,-4.637647,-4.526524,-3.60416,-0.804671,-4.376508,-0.084729,0.948381,1.671738,0.509527,...,4.526524,3.60416,0.804671,4.376508,0.084729,-0.948381,-1.671738,-0.509527,-3.389325e-12,0.509527


In [ ]:
with open("output.txt") as file:
    song_names = [line.rstrip() for line in file]

In [ ]:
# let's move to a new folder to keep the WAV files organized
%mkdir wav_files
%cd wav_files
%pwd

/content/wav_files


'/content/wav_files'

In [ ]:
from collections import Counter
import linecache
import os
import tracemalloc

def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))


In [ ]:
song_names_xeno = []
for song in song_names:
    if song[0:2] == "XC":
        song_names_xeno.append(song)

In [ ]:
results = {}
count = 0
leng = len(song_names_xeno)

for song in song_names_xeno:
    #tracemalloc.start()

    count = count + 1
    #print(song + " RAM: ", psutil.Process(pid).memory_info().rss / (1024**2), "MB" ) #Mem in MB
    coeffs = computeMFCC(song, filterNum = 128)
    if coeffs[1] != -1:
        results[song] = coeffs[0]
        print(count, "/", leng)
        del coeffs
        #snapshot = tracemalloc.take_snapshot()
        #display_top(snapshot)
        #print("--- DONE --- \n")

        #print("Error code -1 returned")
        #snapshot = tracemalloc.take_snapshot()
        #display_top(snapshot)
        #print("--- DONE --- \n")

1 / 3882
2 / 3882
3 / 3882
4 / 3882
5 / 3882
6 / 3882


<ipython-input-26-18a742631681>:72: RuntimeWarning: divide by zero encountered in log10
  audio_log = 10.0 * np.log10(audio_filtered)


7 / 3882
8 / 3882
9 / 3882
10 / 3882
11 / 3882
12 / 3882
13 / 3882
14 / 3882
15 / 3882
16 / 3882
17 / 3882
18 / 3882
19 / 3882
20 / 3882
21 / 3882
22 / 3882
23 / 3882
24 / 3882
not mono 2
25 / 3882
not mono 2
26 / 3882
not mono 2
27 / 3882
not mono 2
28 / 3882
not mono 2
29 / 3882
not mono 2
30 / 3882
not mono 2
31 / 3882
not mono 2
32 / 3882
not mono 2
33 / 3882
not mono 2
34 / 3882
35 / 3882
36 / 3882
37 / 3882
38 / 3882
39 / 3882
40 / 3882
41 / 3882
42 / 3882
43 / 3882
44 / 3882
45 / 3882
46 / 3882
47 / 3882
48 / 3882
49 / 3882
50 / 3882
51 / 3882
52 / 3882
53 / 3882
54 / 3882
55 / 3882
56 / 3882
57 / 3882
58 / 3882
59 / 3882
60 / 3882
not mono 2
61 / 3882


<ipython-input-26-18a742631681>:72: RuntimeWarning: divide by zero encountered in log10
  audio_log = 10.0 * np.log10(audio_filtered)


not mono 2
62 / 3882


...

not mono 2
3804 / 3882
not mono 2
3805 / 3882
not mono 2
3806 / 3882
not mono 2
3807 / 3882
not mono 2
3808 / 3882
not mono 2
3809 / 3882
not mono 2
3810 / 3882
not mono 2
3811 / 3882
not mono 2
3812 / 3882
not mono 2
3813 / 3882
not mono 2
3814 / 3882
not mono 2
3815 / 3882
not mono 2
3816 / 3882
not mono 2
3817 / 3882
not mono 2
3818 / 3882
not mono 2
3819 / 3882
not mono 2
3820 / 3882
not mono 2
3821 / 3882
not mono 2
3822 / 3882
not mono 2
3823 / 3882
not mono 2
3824 / 3882
not mono 2
3825 / 3882
not mono 2
3826 / 3882
not mono 2
3827 / 3882
not mono 2
3828 / 3882
not mono 2
3829 / 3882
not mono 2
3830 / 3882
not mono 2
3831 / 3882
not mono 2
3832 / 3882
not mono 2
3833 / 3882
not mono 2
3834 / 3882
not mono 2
3835 / 3882
not mono 2
3836 / 3882
not mono 2
3837 / 3882
not mono 2
3838 / 3882
not mono 2
3839 / 3882
not mono 2
3840 / 3882
not mono 2
3841 / 3882
not mono 2
3842 / 3882
not mono 2
3843 / 3882
not mono 2
3844 / 3882
not mono 2
3845 / 3882
not mono 2
3846 / 3882
not mono 2


<ipython-input-26-18a742631681>:72: RuntimeWarning: divide by zero encountered in log10
  audio_log = 10.0 * np.log10(audio_filtered)


3880 / 3882


<ipython-input-26-18a742631681>:72: RuntimeWarning: divide by zero encountered in log10
  audio_log = 10.0 * np.log10(audio_filtered)


3881 / 3882
3882 / 3882


In [ ]:
df = pd.DataFrame(results).T
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
XC113545-C304_D01.mp3,-112.424644,3.669452,-9.344393,0.368146,-3.065483,1.477477,-1.491879,0.964105,-0.266210,0.842624,...,-9.344393,3.669452,-158.992456,3.669452,-9.344393,0.368146,-3.065483,1.477477,-1.491879,0.964105
XC113546-C5836_D02.mp3,-128.394095,-8.109159,1.643606,-1.721614,6.578285,3.668080,0.485831,3.190450,-0.613307,3.819558,...,1.643606,-8.109159,-181.576671,-8.109159,1.643606,-1.721614,6.578285,3.668080,0.485831,3.190450
XC113547-C357_D01.mp3,-177.510285,6.310285,-5.041636,4.860886,4.528932,5.786759,1.638032,1.464928,-0.909118,-0.938925,...,-5.041636,6.310285,-251.037452,6.310285,-5.041636,4.860886,4.528932,5.786759,1.638032,1.464928
XC113595-C320_D01.mp3,-112.160947,-5.193795,-2.644363,3.891338,-0.314010,5.532670,2.907106,1.701984,-1.736801,0.766902,...,-2.644363,-5.193795,-158.619533,-5.193795,-2.644363,3.891338,-0.314010,5.532670,2.907106,1.701984
XC113596-C3391_D01.mp3,-127.369801,0.443200,2.291995,12.806234,3.774709,5.318489,-0.867477,1.694896,-2.465154,-2.729120,...,2.291995,0.443200,-180.128100,0.443200,2.291995,12.806234,3.774709,5.318489,-0.867477,1.694896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XC893239-240331_026---Uropelia-campestris.mp3,-29.724795,10.480167,1.655632,2.572706,-20.258279,2.198067,-1.546303,-2.294621,6.821933,1.589022,...,1.655632,10.480167,-42.037208,10.480167,1.655632,2.572706,-20.258279,2.198067,-1.546303,-2.294621
XC893240-240331_028---Nannochordeiles-pusillus.mp3,-42.505570,4.456111,0.089331,21.211386,-5.221518,-0.914210,12.711053,-3.144217,1.786280,3.958931,...,0.089331,4.456111,-60.111953,4.456111,0.089331,21.211386,-5.221518,-0.914210,12.711053,-3.144217
XC899406-Habia-rubica_240428_1209.mp3,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XC899409-Lepidothrix-coronata_240428_1212.mp3,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.dropna()

In [ ]:
%cd ..

/content


In [ ]:
df.to_csv('bird_songs_128_dim.csv')

# **Computing the 3D Embedding of Each Song**
From the 128-dimensional MFCC representations, we use UMAP to first reduce the space to 64 dimensions, and then UMAP again to reduce the space to 3 dimensions.

In [ ]:
!pip install umap-learn

In [ ]:
import numpy as np
import pandas as pd
import umap
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/bird_songs_128_dim.csv')
df.set_index('Unnamed: 0', inplace = True)
df.index.names = ['Song']
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
Song,,,,,,,,,,,,,,,,,,,,,
XC113545-C304_D01.mp3,-112.424644,3.669452,-9.344393,0.368146,-3.065483,1.477477,-1.491879,0.964105,-0.266210,0.842624,...,-9.344393,3.669452,-158.992456,3.669452,-9.344393,0.368146,-3.065483,1.477477,-1.491879,0.964105
XC113546-C5836_D02.mp3,-128.394095,-8.109159,1.643606,-1.721614,6.578285,3.668080,0.485831,3.190450,-0.613307,3.819558,...,1.643606,-8.109159,-181.576671,-8.109159,1.643606,-1.721614,6.578285,3.668080,0.485831,3.190450
XC113547-C357_D01.mp3,-177.510285,6.310285,-5.041636,4.860886,4.528932,5.786759,1.638032,1.464928,-0.909118,-0.938925,...,-5.041636,6.310285,-251.037452,6.310285,-5.041636,4.860886,4.528932,5.786759,1.638032,1.464928
XC113595-C320_D01.mp3,-112.160947,-5.193795,-2.644363,3.891338,-0.314010,5.532670,2.907106,1.701984,-1.736801,0.766902,...,-2.644363,-5.193795,-158.619533,-5.193795,-2.644363,3.891338,-0.314010,5.532670,2.907106,1.701984
XC113596-C3391_D01.mp3,-127.369801,0.443200,2.291995,12.806234,3.774709,5.318489,-0.867477,1.694896,-2.465154,-2.729120,...,2.291995,0.443200,-180.128100,0.443200,2.291995,12.806234,3.774709,5.318489,-0.867477,1.694896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XC881589-Heterocercus-aurantiivertex-Rp-Matamata-29022024-HDA.mp3,-212.563753,3.730505,-3.693060,5.048624,-3.399513,2.554230,0.825179,-1.761160,-0.799665,-1.368757,...,-3.693060,3.730505,-300.610542,3.730505,-3.693060,5.048624,-3.399513,2.554230,0.825179,-1.761160
XC893238-240331_020---Cypsnagra-hirundinacea.mp3,-57.340640,27.955354,-12.910790,-6.234510,6.264205,6.438175,-5.527638,-9.225220,-1.322847,-3.246689,...,-12.910790,27.955354,-81.091910,27.955354,-12.910790,-6.234510,6.264205,6.438175,-5.527638,-9.225220
XC893239-240331_026---Uropelia-campestris.mp3,-29.724795,10.480167,1.655632,2.572706,-20.258279,2.198067,-1.546303,-2.294621,6.821933,1.589022,...,1.655632,10.480167,-42.037208,10.480167,1.655632,2.572706,-20.258279,2.198067,-1.546303,-2.294621


In [ ]:
# reduce to 64 dimensions first
reducer = umap.UMAP(n_components = 64)
scaled_df = StandardScaler().fit_transform(df)
embedding = reducer.fit_transform(scaled_df)

In [ ]:
df_64 = pd.DataFrame(embedding)
df_64.index = df.index
df_64

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
Song,,,,,,,,,,,,,,,,,,,,,
XC113545-C304_D01.mp3,7.387507,10.443632,8.107135,7.973750,9.619689,7.379451,7.452693,8.263961,8.290392,5.833641,...,2.324675,5.773214,4.791446,5.741376,4.744902,4.866849,5.209242,5.779908,5.718445,4.913217
XC113546-C5836_D02.mp3,7.311598,10.698281,8.615342,6.809822,10.551223,7.954082,8.388847,8.267949,7.961405,5.569641,...,2.469321,5.646324,4.756268,5.683822,4.563942,4.945839,5.157930,5.662603,5.822822,4.882309
XC113547-C357_D01.mp3,6.401307,10.458473,7.250256,7.101501,10.186173,7.966839,7.612811,8.300052,8.079047,5.933516,...,2.343506,5.742640,4.766032,5.827486,4.717047,4.937655,5.118827,5.521596,5.738448,4.892690
XC113595-C320_D01.mp3,6.509605,10.492922,7.609274,6.951453,10.039287,7.888436,7.882657,8.181139,7.854994,5.831726,...,2.387456,5.724500,4.757515,5.787179,4.673636,4.927996,5.122452,5.549627,5.792052,4.887069
XC113596-C3391_D01.mp3,6.328786,10.407271,7.027965,7.193534,10.137197,8.002363,7.416300,8.358620,8.214001,6.001605,...,2.323156,5.762112,4.768226,5.855407,4.753506,4.935108,5.119243,5.529820,5.701143,4.906709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XC881589-Heterocercus-aurantiivertex-Rp-Matamata-29022024-HDA.mp3,6.984185,10.256597,7.733573,7.081754,9.680814,8.254450,7.662102,8.165276,7.958005,5.916017,...,2.447950,5.737929,4.759726,5.808734,4.663314,4.958362,5.118176,5.530586,5.764314,4.936157
XC893238-240331_020---Cypsnagra-hirundinacea.mp3,4.843460,10.209941,5.964948,6.535088,9.246273,7.208565,7.326756,8.220279,7.834525,6.183785,...,2.283373,5.786619,4.720909,5.934682,4.922608,4.835649,5.082427,5.666655,5.733884,4.951898
XC893239-240331_026---Uropelia-campestris.mp3,3.455731,10.459031,6.108458,6.991999,7.801466,6.944584,7.173428,7.797314,7.456179,6.027986,...,2.239087,5.964871,4.725192,5.906485,4.962009,4.708613,5.123698,5.702489,5.775581,4.892241


In [ ]:
# reduce to 3 dimensions
reducer = umap.UMAP(n_components = 3)
scaled_df = StandardScaler().fit_transform(df_64)
embedding_3 = reducer.fit_transform(scaled_df)

In [ ]:
df_3 = pd.DataFrame(embedding_3)
df_3.index = df.index
df_3

,0,1,2
Song,,,
XC113545-C304_D01.mp3,14.251409,3.192739,7.275132
XC113546-C5836_D02.mp3,11.759435,2.425643,4.716627
XC113547-C357_D01.mp3,16.528502,3.166834,4.588127
XC113595-C320_D01.mp3,13.704331,3.233704,2.939681
XC113596-C3391_D01.mp3,16.563465,3.147826,4.620661
...,...,...,...
XC881589-Heterocercus-aurantiivertex-Rp-Matamata-29022024-HDA.mp3,13.406098,2.970738,2.784286
XC893238-240331_020---Cypsnagra-hirundinacea.mp3,0.834472,15.738647,4.825076
XC893239-240331_026---Uropelia-campestris.mp3,0.999772,9.918804,3.655380


In [ ]:
songs = df_3.index.to_list()

In [ ]:
df_polyglot = df_3.reset_index()

In [ ]:
'''
RANDOMLY PICKING ANCHOR POINTS (10% OF DATASET)
'''
num_anchors = 345 # 10% of dataset
import random
anchors = random.sample(range(3452), num_anchors)
anchors

[184,
 805,
 2199,
 247,
 1236,
 2801,
 3432,
 2252,
 617,
 3051,
 885,
 3065,
 821,
 1713,
 2926,
 923,
 900,
 959,
 140,
 1661,
 592,
 1761,
 2623,
 2288,
 17,
 3225,
 1569,
 2276,
 245,
 2285,
 745,
 2090,
 3428,
 236,
 1916,
 869,
 1333,
 3160,
 843,
 2200,
 1697,
 2332,
 566,
 3409,
 285,
 397,
 3235,
 978,
 677,
 2832,
 698,
 969,
 2874,
 3364,
 3027,
 70,
 643,
 464,
 2209,
 1193,
 847,
 675,
 196,
 987,
 181,
 1511,
 1251,
 457,
 533,
 758,
 922,
 1794,
 24,
 89,
 2222,
 2038,
 1844,
 1419,
 819,
 3085,
 2370,
 329,
 2183,
 3071,
 1812,
 644,
 3328,
 1568,
 861,
 3398,
 882,
 2542,
 1634,
 2617,
 3289,
 1318,
 2923,
 1650,
 1747,
 547,
 2017,
 1166,
 815,
 3229,
 3341,
 2729,
 1937,
 1199,
 2462,
 1321,
 1783,
 743,
 2591,
 2157,
 3420,
 1253,
 2232,
 48,
 1702,
 2468,
 1653,
 2024,
 1663,
 991,
 3140,
 40,
 1001,
 1,
 1610,
 319,
 1133,
 998,
 1059,
 2233,
 434,
 3357,
 1506,
 1534,
 3176,
 538,
 2135,
 281,
 1409,
 1475,
 1388,
 1280,
 1972,
 2718,
 3070,
 2667,
 440,
 1996,


In [ ]:
'''
GENERATING META-MAIN.TXT FILE FOR POLYGLOT
'''
with open('meta-main.txt', 'w') as the_file:
    the_file.write('0\n')
    for i in range(0, len(anchors)):
        the_file.write(str(anchors[i]) + ' ' + str(songs[anchors[i]]) + '\n')

In [ ]:
'''
GENERATING FULL_DATA FILE FOR POLYGLOT
'''
with open('full_data', 'w') as the_file:
    for i in range(0, len(songs)):
        song = songs[i]
        if i in anchors:
            is_anchor = 1
        else:
            is_anchor = 0
        the_file.write(str(song) + '\n')
        the_file.write(str(embedding_3[i][0]) + ' ' + str(embedding_3[i][1]) + ' ' + str(embedding_3[i][2]) + ' ' + str(is_anchor) + ' \n')

In [ ]:
# convert high dim data to np.array for speed up
df_np = df.to_numpy()
df_np

array([[-1.12424644e+02,  3.66945175e+00, -9.34439301e+00, ...,
         1.47747653e+00, -1.49187904e+00,  9.64105498e-01],
       [-1.28394095e+02, -8.10915907e+00,  1.64360553e+00, ...,
         3.66807995e+00,  4.85831201e-01,  3.19045042e+00],
       [-1.77510285e+02,  6.31028522e+00, -5.04163619e+00, ...,
         5.78675869e+00,  1.63803239e+00,  1.46492808e+00],
       ...,
       [-2.97247950e+01,  1.04801670e+01,  1.65563190e+00, ...,
         2.19806675e+00, -1.54630350e+00, -2.29462068e+00],
       [-4.25055696e+01,  4.45611148e+00,  8.93312597e-02, ...,
        -9.14209811e-01,  1.27110533e+01, -3.14421688e+00],
       [-3.72516986e+01,  2.90570449e+01, -4.76197330e+01, ...,
        -1.14629605e+00, -1.50824568e+00, -5.18971722e+00]])

In [ ]:
'''
COMPUTING EUCLIDEAN SCORES FOR HIGH DIM DATA
'''
from numpy import dot
from numpy.linalg import norm

# np.linalg.norm(a-b)
anchor_euc_scores = {}
for i in range(0, len(anchors)):
    print(str(i) + " done!")
    current_index = anchors[i]
    current_point = df_np[current_index]
    euc_sim_scores = []
    for j in range(0, len(songs)):
        other_point = df_np[j]
        euc_sim = 1 / (1 + np.linalg.norm(current_point - other_point))
        euc_sim_scores.append([j, euc_sim * 100000])
    euc_sim_scores = sorted(euc_sim_scores , key=lambda k: [k[1], k[0]], reverse=True)
    anchor_euc_scores[current_index] = euc_sim_scores[1:]

0 done!
1 done!
2 done!
3 done!
4 done!
5 done!
6 done!
7 done!
8 done!
9 done!
10 done!
11 done!
12 done!
13 done!
14 done!
15 done!
16 done!
17 done!
18 done!
19 done!
20 done!
21 done!
22 done!
23 done!
24 done!
25 done!
26 done!
27 done!
28 done!
29 done!
30 done!
31 done!
32 done!
33 done!
34 done!
35 done!
36 done!
37 done!
38 done!
39 done!
40 done!
41 done!
42 done!
43 done!
44 done!
45 done!
46 done!
47 done!
48 done!
49 done!
50 done!
51 done!
52 done!
53 done!
54 done!
55 done!
56 done!
57 done!
58 done!
59 done!
60 done!
61 done!
62 done!
63 done!
64 done!
65 done!
66 done!
67 done!
68 done!
69 done!
70 done!
71 done!
72 done!
73 done!
74 done!
75 done!
76 done!
77 done!
78 done!
79 done!
80 done!
81 done!
82 done!
83 done!
84 done!
85 done!
86 done!
87 done!
88 done!
89 done!
90 done!
91 done!
92 done!
93 done!
94 done!
95 done!
96 done!
97 done!
98 done!
99 done!
100 done!
101 done!
102 done!
103 done!
104 done!
105 done!
106 done!
107 done!
108 done!
109 done!
110 done!


In [ ]:
'''
SAVING EUCLIDEAN SCORES
'''
from google.colab import drive
drive.mount('/content/drive')
import os
!mkdir bird_songs_euclidean
os.chdir('bird_songs_euclidean')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/bird_songs_euclidean


In [ ]:
from google.colab import files

for anchor in anchors:
    filename = str(anchor)+'.txt'
    with open(filename, 'w') as the_file:
        for score in anchor_euc_scores[anchor]:
            # note that score is of the form: [index, euclidean]
            the_file.write(str(score[0]) + ', ' + str(score[1]) + '\n')

In [ ]:
os.chdir('..')
!pwd
!cp -r "bird_songs_euclidean" "drive/MyDrive/"

/content


# **Computing Parallel Coordinates Features**


1.   Latitude
2.   Longitude
3.   Spectral Centroid
4.   Zero Crossing Rate

This is just an initial list. Future work would focus on creating a larger list + choosing better features.



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/argentina_mexico_ecuador_colombia_brazil_and_peru.csv")

<ipython-input-21-8bf6990abf0f>:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/argentina_mexico_ecuador_colombia_brazil_and_peru.csv")


In [ ]:
df = pd.read_csv("/content/birds.csv")
df

,Cat. Nr.,Common Name,Length,Recordist,Date,Time,Country,Location,Elevation,Type,Remarks,Quality,MP3 Link
0,XC228210,Blue-crowned Manakin,0:20,NaN,2006-07-22,15:00,Brazil,"Sao Gabriel da Cachoeira area, Amazonas",NaN,call,ID certainty 80%. (Archiv. tape 393 side A tra...,B,//xeno-canto.org/sounds/uploaded/OOECIWCSWV/XC...
1,XC120096,Blue-crowned Manakin,0:49,NaN,2012-10-30,08:30,Brazil,"Manicoré, Amazonas",60,"call, song",[sono],B,//xeno-canto.org/sounds/uploaded/TGBFXDVERJ/XC...
2,XC200163,Blue-crowned Manakin,0:42,NaN,1998-12-20,09:00,Brazil,"São Gabriel da Cachoeira, Amazonas",70,"call, song","left bank of rio Negro - terra firme forest, w...",C,//xeno-canto.org/sounds/uploaded/DGVLLRYDXS/XC...
3,XC119434,Blue-crowned Manakin,0:34,NaN,2010-06-06,10:00,Brazil,"Careiro, Amazonas",20,song,Recording (not its ID) has been discussed.,C,//xeno-canto.org/sounds/uploaded/TGBFXDVERJ/XC...
4,XC8330,Blue-crowned Manakin,0:16,NaN,2006-09-19,15:30,Peru,NaN,300,call?,"A single bird, at least no others heard.",C,//xeno-canto.org/sounds/uploaded/SNUDCZVUQK/Le...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,XC38735,Black-tailed Tityra,0:13,NaN,2009-07-23,?:?,Brazil,"Novo Airao, Amazonas",?,?,natural recording,C,//xeno-canto.org/sounds/uploaded/GKADEITXTH/TI...
4716,XC202742,Ruddy-tailed Flycatcher,0:22,NaN,2009-03-10,?,Venezuela,"Junglaven Lodge, Amazonas",120,call,[sono],A,//xeno-canto.org/sounds/uploaded/VROUALCTKG/XC...
4717,XC200253,Ruddy-tailed Flycatcher,0:14,NaN,1998-12-19,10:36,Brazil,"São Gabriel da Cachoeira, Amazonas",70,song,"trail on right bank of rio Negro, just below r...",A,//xeno-canto.org/sounds/uploaded/DGVLLRYDXS/XC...
4718,XC486893,Amazonian Royal Flycatcher,0:48,NaN,2008-03-12,14:02,Brazil,"Rio Tapajós,Apuí,Amazonas",112,call,DCB 617-6g,B,//xeno-canto.org/sounds/uploaded/YOXWKYFZAP/XC...


In [ ]:
temp["MP3 Link"] = temp["MP3 Link"].apply(lambda x: "https:" + str(x))

In [ ]:
temp = df[df['Cat. Nr.'].isin(song_ids)][['Cat. Nr.', 'Common Name', 'Length','Location', 'MP3 Link']]

In [ ]:
temp.to_csv("metadata-secondary.csv", index=False)

In [ ]:
file_names_df = df["file-name"].apply(lambda x: str(x).replace(" ", "%20"))
file_names_df = file_names_df.to_list()
file_names_df

['XC754598-18-09-2022-8e21-Papa-taoca-do-tapajos.mp3',
 'XC695999-STE-037%20Pyriglena%20similis.mp3',
 'XC529078-papa.mp3',
 'XC246018-PYRLNT13.mp3',
 'XC214239-SALCOE12.mp3',
 'XC178515-Thamnophilidae_Pyriglena_leuconota_05_08_2013_H08_14_Rosário_Oeste.mp3',
 'XC174050-PAPA%20TAOCA%20-%20ROSARIO%20OESTE%20-%2005-08-13%20(1).mp3',
 'XC171181-Pyriglena%20leuconota_Jauru_1425.mp3',
 'White-backed%20Fire-eye%20song%208-19-11.mp3',
 'PYRLNT12.mp3',
 'PYRLNT04.mp3',
 'White-backed%20Fire-eye2009-8-23-1part1.mp3',
 'XC754583-18-09-2022-8e13-Papa-taoca-do-tapajos.mp3',
 'XC754581-18-09-2022-8e10-Papa-taoca-do-tapajos-e-formigueiro-de-cauda-baia-(Pyriglena-similis).mp3',
 'XC754579-18-09-2022-8e6-Papa-taoca-do-tapajos.mp3',
 'XC708374-MixPre-130%20possivel%20C%20diamantinensis.mp3',
 'XC580956-2006-06-08-White-backed%20Fire-eye%20song-SOlmstead.mp3',
 'XC385745-Pyriglena_leuconota_Basileuterus_hypoleucus_Dromococcyx_phasianellus_M6_Trans_2-3_Jangada_MT.mp3',
 'XC333327-UNID0478.mp3',
 'XC24602

In [ ]:
with open("output.txt") as file:
    song_names = [line.rstrip() for line in file]

song_names_xeno = []
for song in song_names:
    if song[0:2] == "XC":
        song_names_xeno.append(song)

In [ ]:
song_names_xeno

['XC113545-C304_D01.mp3',
 'XC113546-C5836_D02.mp3',
 'XC113547-C357_D01.mp3',
 'XC113595-C320_D01.mp3',
 'XC113596-C3391_D01.mp3',
 'XC113598-C5807_D01.mp3',
 'XC113599-C328_D01%20.mp3',
 'XC113603-C336_D01.mp3',
 'XC113604-C384_D01%20.mp3',
 'XC113606-C1947%20_D01.mp3',
 'XC113607-C384_D03.mp3',
 'XC113629-C2939_D01.mp3',
 'XC113630-C2933_D01.mp3',
 'XC113634-C2905_D01.mp3',
 'XC113797-C3386%20_D01.mp3',
 'XC115770-Plain-tailed%20Wren_schulenbergi2012-10-28-1.mp3',
 'XC115771-Russet-mantled%20Softtail2012-10-29-1.mp3',
 'XC115772-Neblina%20Tapaculo2012-10-29-2.mp3',
 'XC115773-Neblina%20Tapaculo2012-10-29-4.mp3',
 'XC115774-Neblina%20Tapaculo2012-10-29-5.mp3',
 'XC115775-Blackish%20Tapaculo2012-10-29-1.mp3',
 'XC115935-Pale-billed%20Antpitta2012-10-28-1-1.mp3',
 'XC115936-Smoky%20Bush-Tyrant2012-10-29-1.mp3',
 'XC116693-Rusty-tinged%20Antpitta2012-10-29-2.mp3',
 'XC117313-Track%2016.mp3',
 'XC117474-35%20Audio%20Track.mp3',
 'XC117969-1%20Audio%20Track.mp3',
 'XC117970-2%20Audio%20Tr

In [ ]:
song_names_xeno[101].split('-')[0]

'XC119154'

In [ ]:
song_ids = []
count = 0
for song_name in song_names_xeno:
    song_ids.append(song_name.split('-')[0])
    count += 1
song_ids

['XC113545',
 'XC113546',
 'XC113547',
 'XC113595',
 'XC113596',
 'XC113598',
 'XC113599',
 'XC113603',
 'XC113604',
 'XC113606',
 'XC113607',
 'XC113629',
 'XC113630',
 'XC113634',
 'XC113797',
 'XC115770',
 'XC115771',
 'XC115772',
 'XC115773',
 'XC115774',
 'XC115775',
 'XC115935',
 'XC115936',
 'XC116693',
 'XC117313',
 'XC117474',
 'XC117969',
 'XC117970',
 'XC117971',
 'XC117972',
 'XC117973',
 'XC117974',
 'XC117975',
 'XC117976',
 'XC117994',
 'XC117995',
 'XC117997',
 'XC117998',
 'XC117999',
 'XC118422',
 'XC118467',
 'XC118468',
 'XC118469',
 'XC118470',
 'XC118475',
 'XC118476',
 'XC118478',
 'XC118905',
 'XC118906',
 'XC118907',
 'XC118908',
 'XC118909',
 'XC118911',
 'XC118912',
 'XC118913',
 'XC118967',
 'XC118969',
 'XC118972',
 'XC118975',
 'XC118978',
 'XC119110',
 'XC119111',
 'XC119112',
 'XC119113',
 'XC119114',
 'XC119115',
 'XC119116',
 'XC119117',
 'XC119118',
 'XC119119',
 'XC119120',
 'XC119121',
 'XC119122',
 'XC119123',
 'XC119124',
 'XC119125',
 'XC119126',

In [ ]:
prepend = ""

In [ ]:
### Checking if all songs are in df
len(list(set(file_names_df) & set(song_names_xeno)))

# So only 2751/3882 will have latitude and longitude data...

2751